## 1.导入包

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import re
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

np.set_printoptions(threshold=np.inf)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [2]:
"""
网格搜索(Grid Search)和Optuna，找到模型的最佳超参数组合
网格搜索适用于超参数空间较小、离散且较少的情况，而Optuna适用于超参数空间较大、连续或离散且较多的情况
下面要做的事情：
1.换新的面心值标签，现在数据过拟合，训练集下降但是测试集上升或者波动
2.考虑正则化或者假如droput层来防止过拟合
3.考虑数据预处理中采用数据标准化，让数据均匀分布
4.用不用考虑损失函数，学习率,epoch,adam优化器及其四个参数，的修改，模型用不用再添加几层让模型变复杂些（batch-size越大，训练越快，不影响准确率）
5.早停法（Early Stopping）：在训练过程中监控验证集上的性能，一旦性能停止改善，在一定epoch后停止训练，并保存模型，以防止过拟合。可以参照外国那案例
6.数据集的比例，不一定4：1，也可以95：5，当数据集足够大时，这样可以增加训练集数量
"""

'\n网格搜索(Grid Search)和Optuna，找到模型的最佳超参数组合\n网格搜索适用于超参数空间较小、离散且较少的情况，而Optuna适用于超参数空间较大、连续或离散且较多的情况\n下面要做的事情：\n1.换新的面心值标签，现在数据过拟合，训练集下降但是测试集上升或者波动\n2.考虑正则化或者假如droput层来防止过拟合\n3.考虑数据预处理中采用数据标准化，让数据均匀分布\n4.用不用考虑损失函数，学习率,epoch,adam优化器及其四个参数，的修改，模型用不用再添加几层让模型变复杂些（batch-size越大，训练越快，不影响准确率）\n5.早停法（Early Stopping）：在训练过程中监控验证集上的性能，一旦性能停止改善，在一定epoch后停止训练，并保存模型，以防止过拟合。可以参照外国那案例\n6.数据集的比例，不一定4：1，也可以95：5，当数据集足够大时，这样可以增加训练集数量\n'

## 2.加载数据

"""
1.功能：
通过加载data和label文件，然后继续训练和预测。
定义了一个6层卷积神经网络模型。每个卷积层后面跟着一个 ReLU 激活函数。第七层只有卷积，没有relu。
输入数据n*64*64*2,这里的一个样本64*64可以看成一个图片格式（在此次任务中是速度，两者类似）
输出是n*64*64*4
"""
"""txt保存为numpy格式发现可以减少存储大小，约缩小成1/4
5.9G	./all_data.npy
12G	./all_label.npy
27G	./data_64x64x2.txt
53G	./label_a_2x64x65x2.txt
"""

In [3]:
# 直接加载npy文件为numpy格式
all_data = np.load('./data/all_data.npy')
# #直接加载npy文件为numpy格式,注意标签是面心值，不是a值
all_label = np.load('./data/all_centerFace_label.npy')

all_data = torch.tensor(all_data).float()
all_label = torch.tensor(all_label).float()


NameError: name 'all_data' is not defined

## 3.构建模型

In [4]:

#神经网络模型
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, num_output_channels, dropout):#加了 dropout
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(2, 64, kernel_size=3, stride=1, padding=1)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=dropout)  # 添加dropout层

        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(p=dropout)  # 添加dropout层

        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        self.dropout3 = nn.Dropout(p=dropout)  # 添加dropout层

        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        self.dropout4 = nn.Dropout(p=dropout)  # 添加dropout层

        self.conv5 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu5 = nn.ReLU()
        self.dropout5 = nn.Dropout(p=dropout)  # 添加dropout层

        self.conv6 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu6 = nn.ReLU()
        self.dropout6 = nn.Dropout(p=dropout)  # 添加dropout层

        self.conv7 = nn.Conv2d(64, num_output_channels, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.dropout1(x)  # dropout层应用于卷积层之后

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.dropout2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.dropout3(x)

        x = self.conv4(x)
        x = self.relu4(x)
        x = self.dropout4(x)

        x = self.conv5(x)
        x = self.relu5(x)
        x = self.dropout5(x)

        x = self.conv6(x)
        x = self.relu6(x)
        x = self.dropout6(x)

        x = self.conv7(x)

        return x




In [5]:
"""
早停法是一种被广泛使用的方法，在很多案例上都比正则化的方法要好。
其基本含义是在训练中计算模型在验证集上的表现，
当模型在验证集上的表现开始下降的时候，停止训练，这样就能避免继续训练导致过拟合的问题。
"""

'\n早停法是一种被广泛使用的方法，在很多案例上都比正则化的方法要好。\n其基本含义是在训练中计算模型在验证集上的表现，\n当模型在验证集上的表现开始下降的时候，停止训练，这样就能避免继续训练导致过拟合的问题。\n'

In [6]:
save_model_path = './model/model_origin_1000_zaoting_02.pth'
save_loss_path = './data/lossa/loss_model_1000_zaoting_02.npy'


## 4.模型训练与测试

In [7]:
import optuna
import time
def objective(trial):
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    dropout = trial.suggest_uniform('dropout', 0.1, 0.5)

    num_output_channels = 4
    # 生成数据集
    x_train, x_test, y_train, y_test = train_test_split(all_data, all_label, test_size=0.2)

    # 设置种子数
    seed = 42
    torch.manual_seed(seed)

    # 划分数据集
    trainset = torch.utils.data.TensorDataset(x_train, y_train)
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=batch_size, shuffle=True)

    # 划分数据集
    testset = torch.utils.data.TensorDataset(x_test, y_test)
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=batch_size, shuffle=True)

    # 创建模型实例，并将模型移动到GPU设备上进行计算
    net = Net(num_output_channels,dropout).to(device)

    # 加速训练：如果有多个GPU，则使用DataParallel模块
    if torch.cuda.device_count() > 1:
        net = nn.DataParallel(net)
        print("采用DataParallel加速，device_count个数为：",str(torch.cuda.device_count()))

    # 定义损失函数为均方误差
    criterion = nn.MSELoss()

    # 获取要调优的超参数值
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    optimizer = getattr(optim, optimizer_name)(net.parameters(), lr=lr)
        
#     定义优化器为Adam优化器
#     optimizer = optim.Adam(net.parameters(), lr=lr)

    # 训练模型
    num_epochs = 20
    train_loss = []
    test_loss = []
    
    best_loss = float('inf')  # 初始化最佳验证集损失值为正无穷大
    patience = 10  # 设置连续多少次验证集损失值不下降时停止训练
    count = 0  # 记录连续不下降次数
    
    start_time =  time.time()
    for epoch in range(num_epochs):
        running_train_loss = 0.0
        net.train()#加了这个
        for i, (inputs, labels) in enumerate(trainloader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()

        train_loss.append(running_train_loss)
        
        running_test_loss = 0.0
        net.eval()
        with torch.no_grad():
            for i, (inputs, labels) in enumerate(testloader):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = net(inputs)
                loss = criterion(outputs, labels)
                running_test_loss += loss.item()
        
            
            test_loss.append(running_test_loss)
            print("已完成第：", str(epoch+1), "个epoch! Train Loss:", running_train_loss, "Test Loss:", running_test_loss)

            # 早停法
            """
            如果<连续多个 epoch> 的<验证集损失值>都没有<下降>，即验证集损失值不再降低，那么就会认为模型已经过拟合或者无法继续改善。
            这时，训练会提前停止，并保存当前的模型参数
            """
            if running_test_loss < best_loss:

                best_loss = running_test_loss
                count = 0 #连续十次测试集的epoch loss不下降，故只要又一次下降，就清零重新计算
            else:
                count += 1
                if count >= patience:
                    print(f"验证集损失值连续{patience}次不下降，停止训练！")
                    break

#             if epoch==500:
#                 torch.save(net, './model/model_origin_500_02.pth')
    #     注意
    #缺少保存loss代码
    #急停法和optuna能不能同时用，看那个外国视频代码
    
    end_time = time.time()
    process_time = end_time - start_time
    print(f"模型训练和测试共用了: {process_time} 秒！")
    print('all of tasks Finished')
    
    # 保存整个模型
    torch.save(net, save_model_path)
    
    if trial.should_prune():
        raise optuna.exceptions.TrialPruned()

    # 返回验证集上的最终损失作为目标值
    return running_test_loss



In [ ]:
"""
使用study.optimize方法来运行Optuna的超参数搜索过程，设置n_trials参数为搜索的迭代次数。
搜索结束后，可以通过study.best_trial获取最佳的超参数组合，打印出最佳超参数以及最小化的损失函数值。
"""
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))
    

[I 2023-07-20 17:06:37,386] A new study created in memory with name: no-name-5a080d2c-deef-42c1-b490-895fae9672fd
/tmp/ipykernel_3765038/3893867180.py:5: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout = trial.suggest_uniform('dropout', 0.1, 0.5)


采用DataParallel加速，device_count个数为： 4


/tmp/ipykernel_3765038/3893867180.py:37: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)


已完成第： 1 个epoch! Train Loss: 1392.8881187438965 Test Loss: 339.3895390033722
已完成第： 2 个epoch! Train Loss: 1324.8325445652008 Test Loss: 328.97430324554443
已完成第： 3 个epoch! Train Loss: 1294.918596982956 Test Loss: 318.3123745918274
已完成第： 4 个epoch! Train Loss: 1268.1907320022583 Test Loss: 309.579740524292
已完成第： 5 个epoch! Train Loss: 1253.023736000061 Test Loss: 305.989667057991
已完成第： 6 个epoch! Train Loss: 1244.4158878326416 Test Loss: 303.2557336091995
已完成第： 7 个epoch! Train Loss: 1238.0536451339722 Test Loss: 301.8176211118698
已完成第： 8 个epoch! Train Loss: 1233.2023161649704 Test Loss: 300.29429483413696
已完成第： 9 个epoch! Train Loss: 1228.9670069217682 Test Loss: 299.8783633708954
已完成第： 10 个epoch! Train Loss: 1225.885414838791 Test Loss: 298.769523024559
已完成第： 11 个epoch! Train Loss: 1222.875124335289 Test Loss: 296.97609436511993
已完成第： 12 个epoch! Train Loss: 1220.5153963565826 Test Loss: 296.30419397354126
已完成第： 13 个epoch! Train Loss: 1217.7758724689484 Test Loss: 295.4649648666382
已完成第： 14 个e

[I 2023-07-20 17:20:14,350] Trial 0 finished with value: 291.7751262187958 and parameters: {'batch_size': 256, 'dropout': 0.3409096517100847, 'lr': 0.0004231127631425164, 'optimizer': 'Adam'}. Best is trial 0 with value: 291.7751262187958.


采用DataParallel加速，device_count个数为： 4
已完成第： 1 个epoch! Train Loss: 2494.1426713466644 Test Loss: 568.3339703083038
已完成第： 2 个epoch! Train Loss: 1795.6325256824493 Test Loss: 347.9686965942383
已完成第： 3 个epoch! Train Loss: 1416.697437763214 Test Loss: 345.88529777526855
已完成第： 4 个epoch! Train Loss: 1408.2382943630219 Test Loss: 345.1840386390686
已完成第： 5 个epoch! Train Loss: 1403.9834320545197 Test Loss: 344.74241495132446
已完成第： 6 个epoch! Train Loss: 1400.9367306232452 Test Loss: 344.37344193458557


In [ ]:
print(all_data.shape)
print(all_label.shape)


In [ ]:
train_loss,test_loss = np.load(save_loss_path)
print(train_loss)

# 5.可视化loss

In [ ]:
import matplotlib.pyplot as plt
# 创建 x 轴数据，假设为 epoch 数
epoch = np.arange(1, len(train_loss) + 1)

train_loss = [round(int(a)/(all_data.shape[0]*0.8), 5) for a in train_loss]
print(train_loss)
# 绘制训练损失曲线
plt.plot(epoch, train_loss, 'b', label='Train Loss')

# 关闭科学计数法
plt.gca().get_yaxis().get_major_formatter().set_scientific(False)

# 设置y轴范围下限为0
# plt.ylim(bottom=0)

# 设置图表标题和轴标签
plt.title('Train Loss')
plt.xlabel('epoch')
plt.ylabel('Loss')

# 添加图例
plt.legend()

# 显示图表
plt.show()

In [ ]:
# 创建 x 轴数据，假设为 epoch 数
epoch = np.arange(1, len(test_loss) + 1)
print(test_loss)
test_loss = [round(int(a)/(all_data.shape[0]*0.2), 5) for a in test_loss]


# 绘制测试损失曲线
plt.plot(epoch, test_loss, 'r', label='Test Loss')

# 设置图表标题和轴标签
plt.title('Test Loss')
plt.xlabel('epoch')
plt.ylabel('Loss')

# 添加图例
plt.legend()

# 显示图表
plt.show()

In [ ]:
# from torchsummary import summary
# # 将模型移动到适当的设备
# model = model.to(device)

# # 打印模型的概要信息
# summary(model,input_size=(2, 64, 64))

## 6.预测

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 加载模型
model = torch.load(save_model_path, map_location=device)# from torchsummary import summary
# # 将模型移动到适当的设备

model = model.to(device)


In [ ]:
# 设置模型为评估模式
model.eval()

# 输入数据进行预测
input_data =all_data[-10]  # 你的输入数据


#调整输入input的维度顺序,作为E，用于下面change_Label_to_a中(E-A)/(B-A)得到a值
matrix_64 = input_data.cpu()
matrix_64 = matrix_64.permute(1,2,0)  
print(matrix_64.shape)


# 转换为四维
input_data = input_data.unsqueeze(0)#用实际数据，数据格式为(1,2, 64, 64)，不能为2x64x64
# input_data = torch.randn(1,2, 64, 64)
print(input_data.shape)
input_tensor = input_data.to(device)

with torch.no_grad():
    output = model(input_tensor)##如果报错的话需要把网络的设计加上，里面涉及model(input)

# 打印预测结果
print(output)

# 8.转换格式(label转为最终的weights)

In [ ]:
#将label面心值转为a值
def change_Label_to_a(all_vertical_edge_centers,all_horizontal_edge_centers):
    a_vertical = np.zeros((64, 65, 2))
    # a_vertical = np.random((64, 65, 2))
    a_horizontal = np.zeros((64, 65, 2))
    # a_horizontal = np.random.random((64, 65, 2))


    # 21. 求a:   横着的边，分两种情况，边缘（对称的）和非边缘的边.这里matrix_64要行列互换，因为横着时面心值是一列一列求得，竖着时是一行一行求的。
    for i in range(64):
        for j in range(65):
            if j == 0 or j == 64:
                # a_horizontal[i, j] = (all_horizontal_edge_centers[i, 0] - matrix_64[i, 0]) / (matrix_64[i, 63] - matrix_64[i, 0])#换之前
                a_horizontal[i, j] = (all_horizontal_edge_centers[i, 0] - matrix_64[0, i]) / (
                    matrix_64[63, i] - matrix_64[0, i])
            else:
                # aA=(1-a)B=E   a = (E-B)/(A-B) 其中：A为matrix_64[i,j]，B为matrix_64[i,j+1]
                # a_horizontal[i, j] = (all_horizontal_edge_centers[i, j] - matrix_64[i, j]) / (matrix_64[i, j-1] - matrix_64[i, j])
                a_horizontal[i, j] = (all_horizontal_edge_centers[i, j] - matrix_64[j, i]) / (
                    matrix_64[j - 1, i] - matrix_64[j, i])

    # 22. 求a:   竖着的边，分两种情况，边缘（对称的）和非边缘的边
    for i in range(64):
        for j in range(65):
            if j == 0 or j == 64:
                a_vertical[i, j] = (all_vertical_edge_centers[i, 0] - matrix_64[i, 0]) / (
                    matrix_64[i, 63] - matrix_64[i, 0])
            else:
                # aA=(1-a)B=E   a = (E-B)/(A-B) 其中：A为matrix_64[i,j]，B为matrix_64[i,j+1]
                # 2.错误matrix_64[i, j-1]) / (改成matrix_64[i, j]) / (
                a_vertical[i, j] = (all_vertical_edge_centers[i, j] - matrix_64[i, j]) / (
                    matrix_64[i, j - 1] - matrix_64[i, j])

    # # 若最终a对应的矩阵里面出现无穷，则将其替换为0.5.解决了分母为0的问题
    # a_vertical[np.isinf(a_vertical)] = 0.5
    # a_horizontal[np.isinf(a_horizontal)] = 0.5
    #这里64x65x2截成64x64x2,因为边框对称时值相同
    
    a_vertical = torch.tensor(a_vertical[:,:64,:])
    a_horizontal = torch.tensor(a_horizontal[:,:64,:])

    print(a_vertical.shape)
    print(a_horizontal.shape)
    return  a_vertical,a_horizontal

In [ ]:
#返回一个列表，里面嵌套两个子列表，第一个子列表存放的是内部的a值，第二个子列表存放的是边框的a值，
#且顺序为上（左到右），下（左到右），左（下到上），右（下到上）
def conversion_format(output):
    # 调整output的维度顺序
    output = output.permute(0,2,3,1)  
    print(output.shape)
    output = output[0]
    print(output.shape)
    # print(output)

    #将输出output拆成两个面心值
    all_vertical_edge_centers = output[:, :, 0:2].cpu()
    all_horizontal_edge_centers = output[:, :, 2:4].cpu()
    print(all_vertical_edge_centers.shape)
    print(all_horizontal_edge_centers.shape)

    
    a_vertical,a_horizontal  = change_Label_to_a(all_vertical_edge_centers,all_horizontal_edge_centers)    
    
    # 将前两个元素相加除以二得到一个元素(x+y/)2
    avg_vertical = (a_vertical[:, :, 0] + a_vertical[:, :, 1]) / 2
    # 将后两个元素相加除以二得到另一个元素
    avg_horizontal = (a_horizontal[:, :, 0] + a_horizontal[:, :, 1]) / 2

    # 重新组合成新的形状为(64, 64, 2)的张量
    new_avg_a_output = torch.stack([avg_vertical, avg_horizontal], dim=2)
    # 打印转换后的数据形状
    print(new_avg_a_output.shape)

    #返回两个求完平均的面心值,包括两个64*64矩阵，矩阵是求完平均后的a值，一个竖着的，一个横着的
    vertical_1d = new_avg_a_output[:, :, 0]
    horizontal_1d = new_avg_a_output[:, :,1]
    print(vertical_1d.shape)
    print(horizontal_1d.shape)
    
    
    border = []#存所有边框，四个边框
    left_border=[]#存左边框
    bottom_border = []#存下边框
    inner = []#存内部的面心值

    #下面将2个64x64面心值变换格式，返回指定的格式result
    for i in range(len(vertical_1d)): #两个for循环等价于for i in range(64):
        for j in range(len(vertical_1d[i])):
            if j ==0:#j=0添加边框
                #添加左边框
                left_border.append(vertical_1d[i][0])
                #添加下边框
                bottom_border.append(horizontal_1d[i][0])
            else: 
                if i !=  63:#当竖着的最后一行时，上面没有对应的横着的
                    inner.append(vertical_1d[i][j])#竖着的
    #                 print("{j-1},{i+1}",j-1,i+1)
                    inner.append(horizontal_1d[j-1][i+1])   #再横着的   
                    if j == 63:#如果j=63的话，还需要再加入最后一列的横着的边
                        inner.append(horizontal_1d[63][i+1])#当i=63,横着的加最后一列 
                else:#if i ==63 :
                    inner.append(vertical_1d[63][j])#当i=63,inner最后添加竖着的一行竖线

    inner = [tensor.cpu().numpy().tolist() for tensor in inner]#将一维列表里面的tensor元素转为numpy格式，并返回cpu版本
    # print(inner)
    four_border = [bottom_border,bottom_border,left_border,left_border]#顺序是上（左到右），下（左到右），左（下到上），右（下到上）
    border = [item.numpy().tolist() for sublist in four_border for item in sublist]
    result = [inner,border]
    print(len(result))
    return result



In [ ]:
result  = conversion_format(output)

In [ ]:
print(len(result[0]))
print(len(result[1]))
print(8064+64*2)

In [ ]:
print(result)

In [ ]:
count = sum(1 for num in result[0] if 0 < num < 1)
print(count)
print(count/len(result[0]))